In [33]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os

In [46]:
# data_path=r'D:\PythonDev\MyQuantFinProject\Data\thai-small_2023-now.csv'
data_path='data/WorldOct-No23.csv'
trade_plan_path=r'data/Config-BuySellTrade.xlsx'

TotalBudget=100*1000

#trade_sheet='ThaiNov22-Now23'
trade_sheet='WorldOct-Nov-23'

start_date='2023-10-01'
end_date='2023-11-30'



In [47]:
file_name=trade_sheet
file_name=f'{trade_sheet}'
noFee=False


fee_sheet='Fee'
# data_path=r'D:\PythonDev\MyQuantFinProject\Data\thai_fund.csv'

# excludeFund=['ASP-T12','KKP-TQG']
excludeFund=[]

if noFee==True:
 report_path=f'report/{file_name}(no-fee).xlsx'
else:
 report_path=f'report/{file_name}.xlsx' 
print(report_path)
# trade_sheet='BuySell_Buy_Hold'
# report_path=f'report/BuySell_4ETFFunds.xlsx'


report/WorldOct-Nov-23.xlsx


In [48]:
# if os.path.exists(data_path) and  os.path.exists(data_path):
 
#     print("Change paramter directly on BuySellPortfolio.py prior to running this file.")
#     print(f"Filename : {data_path}")
#     print(f"Trade Plan Sheet Name:{trade_sheet}")
#     print(f"Trad config :{trade_plan_path}")
#     print(f"Simulatite data from {start_date} to {end_date} on report {report_path}")
#     press_y=input(f"Press y=True run trade buy/sell simulation by budget = {TotalBudget}/fund : ") 
#     if press_y.lower()=='y':
#      ok=True
#     else:
#      exit()
# else:
#     raise Exception(f"Not fould {data_path}  or {trade_plan_path}")

# Read Price CSV

In [49]:
df=pd.read_csv(data_path,parse_dates=['Date/Time'],dayfirst=True)
df.columns=['symbol','date','price']
df=df.query('date>=@start_date and date<=@end_date')
df.info()
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
Index: 164 entries, 0 to 163
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   symbol  164 non-null    object        
 1   date    164 non-null    datetime64[ns]
 2   price   164 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 5.1+ KB
  symbol       date  price
0   AAXJ 2023-10-02  63.26
1   AAXJ 2023-10-03  62.49
2   AAXJ 2023-10-04  62.34
3   AAXJ 2023-10-05  62.77
4   AAXJ 2023-10-06  63.75
5   AAXJ 2023-10-09  63.45
6   AAXJ 2023-10-10  64.25
7   AAXJ 2023-10-11  64.68
8   AAXJ 2023-10-12  64.03
9   AAXJ 2023-10-13  63.78


# List Symbol and Plot Chart

In [50]:
list_symbols=list(df['symbol'].unique())
list_symbols=[ f for f in list_symbols if f not in excludeFund ]
list_symbols
# list_symbols=['ACWI']


['AAXJ', 'ACWI', 'QQQ', 'SPY']

In [51]:
# print(list_symbols)
# for  symbol in list_symbols:
#     print(f"=============================={symbol}==============================")
#     dfFund=df.query('symbol==@symbol')
#     plt.figure(figsize=(15, 8))
#     plt.plot(dfFund['date'],dfFund['price'])
#     plt.show()

# Get Trade Data and Fee

In [52]:
# read feeon 
dfAssetInfo=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name=fee_sheet,\
                    index_col='symbol', dtype ={'buy_fee_pct': np.float64,'sell_fee_pct': np.float64})

# check where  fund fee  in portfolio  are avaible?
dfNoFeeInfo= [ item for item in list_symbols if item not in list(dfAssetInfo.index)]
dfAssetInfo=dfAssetInfo.loc[list(list_symbols),:]

if noFee==True:
    dfAssetInfo[['buy_fee_pct','sell_fee_pct']]=0
    
print("fee info")
print(dfAssetInfo) 
# For instance   AAXJ Price=100 but we need to pay 101 $

fee info
        buy_fee_pct  sell_fee_pct region
symbol                                  
AAXJ            1.0           1.0    fif
ACWI            1.0           1.0    fif
QQQ             0.0           0.0    fif
SPY             0.0           0.0    fif


In [53]:
dfPlan=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name=trade_sheet, \
                     parse_dates=['trade_date'],dtype ={'portion_pct': np.float64})

#total buy portion is equal or greater than sell portion
buy_sum_act=dfPlan.query("action=='b'").portion_pct.sum()
sell_sum_act=dfPlan.query("action=='s'").portion_pct.sum()

all_portion=0

In [54]:
if buy_sum_act<sell_sum_act:
  raise Exception("Not allow Buy <= Sell")   
else:
    if buy_sum_act>sell_sum_act:
       print("=============Calculate the last portion to be sellable")
       sellAllAtNow=buy_sum_act-sell_sum_act 
       dfLastPrice=pd.DataFrame(columns=list(dfPlan.columns), data=[ [df['date'].max(),sellAllAtNow,'s']])
       print(dfLastPrice)
       dfPlan=pd.concat([dfPlan,dfLastPrice])
       dfPlan=dfPlan.reset_index(drop=True)

    tradeDateList= [ d for d in list(dfPlan['trade_date']) ]    
    all_portion=buy_sum_act   
    print(f"portion = {all_portion} on")
    
print("==========================Summarize All Trade==========================================")
print(f"No Buy/Sell {dfPlan.groupby(['action']).size()}")
print(dfPlan)

=============Calculate the last portion to be sellable
  trade_date  portion_pct action
0 2023-11-28         50.0      s
portion = 100.0 on
==========================Summarize All Trade==========================================
No Buy/Sell action
b    3
s    2
dtype: int64
  trade_date  portion_pct action
0 2023-10-05         50.0      b
1 2023-11-01         25.0      b
2 2023-11-10         25.0      b
3 2023-11-22         50.0      s
4 2023-11-28         50.0      s


In [55]:
# To make it easier for use to understand portfolio transatoin 
rrr=TotalBudget % all_portion
if rrr !=0 :
  raise Exception(f"Your Budget {TotalBudget} is  not proportional  Portion with {all_portion}")
else:
  print(f"Your Budget {TotalBudget} is  proportional  Portion with {all_portion}")

Your Budget 100000 is  proportional  Portion with 100.0


# Custom Function

In [56]:
# figure out the amount of qty from total fund total value
def get_qty_buy(item):
    xCost=(item['portion_pct']/all_portion)*TotalBudget  
    xPrice=item['price_on_fee']
    qty=xCost/xPrice
    return qty

# figure the amount of  qty out  from total qty
def get_qty_sell(item,TotalQty):  
    qty=(item['portion_pct']/all_portion)*TotalQty
    return qty

In [57]:
def set_fee_by_action(item):
  x=item['symbol']  
  if item['action']=='b':
    fee_type='buy_fee_pct'
  elif  item['action']=='s':
    fee_type='sell_fee_pct'
  try:
     return dfAssetInfo.loc[x,fee_type]
  except Exception as ex:
     print(f"Error : no {x} to get fund fee")
     return 0

In [58]:
dict_dfTran={}
dfSummary=pd.DataFrame(columns=['Symbol','BuyValue','AvgBuyPrice','SellValue','AvgSellPrice','PL','Percent_PL'])

dfSummary
#https://www.w3resource.com/pandas/dataframe/dataframe-apply.php

for symbol in list_symbols:
    print(f"*******************{symbol}*******************")

    dfBS=df[ (df['symbol']==symbol) & (df['date'].isin(tradeDateList))]
    # if len(tradeDateList)==len(dfBS):
        
     # Step1  join price data (filter by sybom and the date whiich you have buy/sell action)   
    dfBS=dfBS.merge(dfPlan,how='inner',left_on='date',right_on='trade_date')
     # remove  redundant data  
    dfBS=dfBS.drop(columns=['trade_date'])     
       
    
     # Step2 calculate fee function
    dfBS['fee']=dfBS.apply(set_fee_by_action,axis=1) # 2.1
    
     # The price that allows you to buy the fund is more expensive than the actual asset price  1%  ,other the other had 
     # Buy price*1.0x=??   
    dfBS['price_on_fee']=dfBS.apply(lambda item : item['price']*(1+ (item['fee']/100)) if item['action']=='b' else item['price']*(1-(item['fee']/100)), axis=1)
    dfBS['price_on_fee']=dfBS['price_on_fee'].round(2) #2.2   
    # print(dfBS)
        
     # Step3  To make it easier to calculate and trading sumary in tearm of qty both buy/sell , both use different way
     # we recommend spliting this dataframe  into 2 dataframe
     #  BuyDataFrame and Sell Date , and we will merge both later  
    
    #3.1 query only buy action
    print("======================Buy=======================")
    buyDF=dfBS.query("action=='b'")   
    
    # 3.2 send the  row to function to calculate qty 
    buyDF['qty']=buyDF.apply(get_qty_buy,axis=1) 
    buyDF['value']=round(buyDF['qty']*buyDF['price_on_fee'],0)
    
    
    # 1 is buy (I will take columne to apply cumsum function to calcaulate cumulative return
    buyDF['action_value']=buyDF['value']
    buyDF['action_qty']=buyDF['qty']
    
    # 3.3 Total qty and value and avg
    TotalQty=buyDF['qty'].sum()
    TotalValue=round(buyDF['value'].sum(),0)
    AvgPrice=round(TotalValue/TotalQty,2)
    
    # print(buyDF[['date','portion_pct','value' ,'price','fee','price_on_fee','qty']])
    
    
    # Step4 follow every step corresponidng to prev step#3
    print("=======================Sell=======================")
    # step#4.1
    sellDF=dfBS.query("action=='s'")  
    
    # ste#4.2
    sellDF['qty']=sellDF.apply(get_qty_sell,axis=1,args=(TotalQty,))
    sellDF['value']=round(sellDF['qty']*sellDF['price_on_fee'],0)
    
    # -1 is sell (I will take columne to apply cumsum function to calcaulate cumulative return
    sellDF['action_value']=sellDF['value']*-1
    sellDF['action_qty']=sellDF['qty']*-1
    
    # print(sellDF[['date' ,'portion_pct','value','price','fee','price_on_fee','qty']])
    

    
    print("======================Merge Buy & Sell======================")
    # step 5  concatenate buy and sell dataframe 
    dfTran=pd.concat([buyDF,sellDF],axis=0)
    dfTran=dfTran.sort_values(by=['date'])
    dfTran[['value','action_value']]=dfTran[['value','action_value']].round(0)

    # step 6 accumulate value and qury over entire data since start buying to last record 
    dfTran['net_value']=dfTran['action_value'].cumsum(axis = 0) 
    dfTran['net_qty']=dfTran['action_qty'].cumsum(axis = 0)  
    dfTran['avg_price']=round( dfTran['net_value']/dfTran['net_qty'],2)
    
    dfTran=dfTran.sort_values(by=['date'])
    # print(dfTran[['date' ,'action','price_on_fee','qty','net_qty','value','net_value','avg_price']])
    print(dfTran[['date' ,'action','price_on_fee','qty','value','avg_price']])
    
    dict_dfTran[symbol]=dfTran
    
    print("=======================Summarize Gain/Loss=======================")
    #step 7 create total buy/sell value , avg prie and profit/loss(value and %) of  all funds 
    # step 7.1 apply sum agg over  value and qty to find avg throughout  the whold dataframe
    SellTotalQty=sellDF['qty'].sum()
    SellTotalValue=round(sellDF['value'].sum(),0)
    SellAvgPrice=round(SellTotalValue/SellTotalQty,0)
    
    print(f"Total Buy {symbol} : qty={TotalQty} and value={TotalValue} at AvgPrice={AvgPrice}")
    print(f"Total Sell {symbol} : qty={SellTotalQty} and value={SellTotalValue} at AvgPrice={SellAvgPrice}")
    

    
    # step #7.2
    PL=round(SellTotalValue-TotalValue,0)
    PCT_PL=round(PL/TotalValue*100,2)
    
    # step# 7.3 
    # you take amont 100000.0 and gain profit103500 from investment on this etf fund 
    summary_data=pd.Series( {'Symbol':symbol,'BuyValue':TotalValue,'AvgBuyPrice':AvgPrice,'SellValue':SellTotalValue,\
                  'AvgSellPrice':SellAvgPrice, 'PL':PL,'Percent_PL':PCT_PL
                 }).to_frame().T
    dfSummary=pd.concat([dfSummary,summary_data],ignore_index=True)
    
    print(dfSummary)

    


*******************AAXJ*******************
======================Buy=======================
=======================Sell=======================
======================Merge Buy & Sell======================
        date action  price_on_fee         qty    value  avg_price
0 2023-10-05      b         63.40  788.643533  50000.0      63.40
1 2023-11-01      b         62.45  400.320256  25000.0      63.08
2 2023-11-10      b         64.50  387.596899  25000.0      63.43
3 2023-11-22      s         65.11  788.280344  51325.0      61.75
4 2023-11-28      s         65.18  788.280344  51380.0       -inf
=======================Summarize Gain/Loss=======================
Total Buy AAXJ : qty=1576.5606885527986 and value=100000.0 at AvgPrice=63.43
Total Sell AAXJ : qty=1576.5606885527986 and value=102705.0 at AvgPrice=65.0
  Symbol  BuyValue AvgBuyPrice SellValue AvgSellPrice      PL Percent_PL
0   AAXJ  100000.0       63.43  102705.0         65.0  2705.0        2.7
*******************ACWI***********

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_26472\2358373929.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_26472\2358373929.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=round(buyDF['qty']*buyDF['price_on_fee'],0)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_26472\2358373929.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [59]:
dfSummary

,Symbol,BuyValue,AvgBuyPrice,SellValue,AvgSellPrice,PL,Percent_PL
0,AAXJ,100000.0,63.43,102705.0,65.0,2705.0,2.7
1,ACWI,100000.0,92.88,104145.0,97.0,4145.0,4.15
2,QQQ,100000.0,362.97,107480.0,390.0,7480.0,7.48
3,SPY,100000.0,427.95,106316.0,455.0,6316.0,6.32


In [67]:
dfPort=dfSummary[['BuyValue','SellValue','PL']].sum().to_frame('All Funds')
dfPort=dfPort.T
dfPort['PercentPL']= (dfPort['PL']/dfPort['BuyValue'] *100)
dfPort

,BuyValue,SellValue,PL,PercentPL
All Funds,400000.0,420646.0,20646.0,5.1615


In [70]:
with pd.ExcelWriter(report_path) as writer:

    dfPort.to_excel(writer,sheet_name='TotalPort',index=False)

    dfSummary=dfSummary.sort_values(by=['Percent_PL'],ascending=False)
    dfSummary.to_excel(writer, sheet_name="Summary",index=False)

    for symbol,tranDF in dict_dfTran.items():
     tranDF=tranDF.drop(columns=['symbol'])   
     tranDF.to_excel(writer, sheet_name=symbol,index=False) 


print("Successfully")

Successfully
